In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
change_type_map = {'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5} 

In [3]:
train_df = gpd.read_file('train.geojson', index_col=0)
test_df = gpd.read_file('test.geojson', index_col=0)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:200: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:200: RuntimeWarning: driver GeoJSON does not support open option INDEX_COL
  return ogr_read(


# <u> Data Preprocessing and Cleaning <u/>

In [4]:
y_train=train_df['change_type'].map(change_type_map) #we convert the target variable to numeric values using the dictionary we created earlier
x_train=train_df.drop(columns=['change_type','index'])#we drop index and change_type columns from the training data
x_test=test_df.drop(columns=['index'])#we drop index column from the test data

In [5]:
numerical_cols = x_train.select_dtypes(include=["number", "bool"]).columns.to_list()
categorical_cols = x_train.select_dtypes(include=["object", "category"]).columns.to_list()

### Geometry features engineering:

In [6]:
def add_geometry_features(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    gdf = gdf.copy()

    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326", allow_override=True)

    gdf_m = gdf.to_crs(epsg=3857)

    area = gdf_m.geometry.area
    perim = gdf_m.geometry.length

    gdf_ll = gdf.to_crs(epsg=4326)
    cent = gdf_ll.geometry.centroid
    bounds = gdf_ll.geometry.bounds 

    gdf["geom_area_m2"] = area
    gdf["geom_perim_m"] = perim
    gdf["geom_compactness"] = (4 * np.pi * area) / (perim**2 + 1e-9)

    gdf["centroid_lon"] = cent.x
    gdf["centroid_lat"] = cent.y

    gdf["bbox_width_deg"]  = bounds["maxx"] - bounds["minx"]
    gdf["bbox_height_deg"] = bounds["maxy"] - bounds["miny"]
    gdf["bbox_area_deg2"]  = gdf["bbox_width_deg"] * gdf["bbox_height_deg"]

    return gdf


In [7]:
#for x_train
x_train = add_geometry_features(x_train)
x_train = x_train.drop(columns=["geometry"])

# for x_test
x_test = add_geometry_features(x_test)
x_test = x_test.drop(columns=["geometry"])

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/shapely/measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/shapely/measurement.py:196: RuntimeWarning: invalid value encountered in length
  return lib.length(geometry, **kwargs)
/var/folders/52/gw_8zl6n65b05xp9tk246k2m0000gn/T/ipykernel_61399/2612037705.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cent = gdf_ll.geometry.centroid
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/shapely/measurement.py:50: RuntimeWarning: invalid value encountered in area
  return lib.area(geometry, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/shapely/measurement.py:196: 

### Date features engineering:

In [8]:
import re

def _detect_date_indices(df):
    date_cols = [c for c in df.columns if re.fullmatch(r"date\d+", c)]
    if not date_cols:
        raise ValueError("Aucune colonne dateX trouvée (date0.. ou date1..)")
    idxs = sorted([int(c.replace("date","")) for c in date_cols])
    return idxs, [f"date{i}" for i in idxs]

def add_temporal_status_image_features(df: pd.DataFrame, drop_raw: bool = True) -> pd.DataFrame:
    """
    - Trie chaque ligne par les vraies dates (NaT à la fin)
    - Construit des features de statuts (ordre + transitions)
    - Construit des features RGB (first/last/deltas) après tri
    - Ajoute aussi date_span_days + deltas entre dates
    """
    df = df.copy()

    idxs, date_cols = _detect_date_indices(df)
    k = len(date_cols)

    status_cols = [f"change_status_date{i}" for i in idxs if f"change_status_date{i}" in df.columns]
    if len(status_cols) != k:

        status_cols = [c for c in df.columns if re.fullmatch(r"change_status_date\d+", c)]
        status_idxs = sorted([int(c.split("date")[-1]) for c in status_cols])
        status_cols = [f"change_status_date{i}" for i in status_idxs]
        k = min(len(date_cols), len(status_cols))
        date_cols, status_cols, idxs = date_cols[:k], status_cols[:k], idxs[:k]

    img_specs = []
    for color in ["red", "green", "blue"]:
        for stat in ["mean", "std"]:
            cols = []
            ok = True
            for i in idxs[:k]:
                c = f"img_{color}_{stat}_date{i}"
                if c in df.columns:
                    cols.append(c)
                else:
                    ok = False
                    break
            if ok:
                img_specs.append((color, stat, cols))


    D = df[date_cols].apply(pd.to_datetime, dayfirst=True, errors="coerce").to_numpy(dtype="datetime64[ns]")
    Di = D.astype("int64")
    nat_mask = (Di == np.iinfo(np.int64).min)
    Di[nat_mask] = np.iinfo(np.int64).max

    order = np.argsort(Di, axis=1)
    Di_sorted = np.take_along_axis(Di, order, axis=1)

    maxv = np.iinfo(np.int64).max
    valid_mask = (Di_sorted != maxv)
    last_idx = valid_mask.sum(axis=1) - 1


    span_days = np.full(len(df), np.nan)
    ok = last_idx >= 0
    span_days[ok] = (Di_sorted[ok, last_idx[ok]] - Di_sorted[ok, 0]) / (24 * 3600 * 1e9)
    df["date_span_days"] = span_days

    for j in range(k - 1):
        a, b = Di_sorted[:, j], Di_sorted[:, j + 1]
        delta = (b - a) / (24 * 3600 * 1e9)
        bad = (a == maxv) | (b == maxv)
        delta[bad] = np.nan
        df[f"delta_sorted_{j}_{j+1}_days"] = delta


    S = df[status_cols].astype(object).to_numpy()
    S_sorted = np.take_along_axis(S, order[:, :k], axis=1)


    Sdf = pd.DataFrame(S_sorted)
    df["status_first"] = Sdf.bfill(axis=1).iloc[:, 0]
    df["status_last"]  = Sdf.ffill(axis=1).iloc[:, -1]

    df = pd.concat([df, pd.get_dummies(df["status_first"], prefix="status_first", dummy_na=True)], axis=1)
    df = pd.concat([df, pd.get_dummies(df["status_last"],  prefix="status_last",  dummy_na=True)], axis=1)


    level_map = {
        "Land Cleared": 0,
        "Construction Started": 1,
        "Materials Dumped": 1,
        "Construction Midway": 2,
        "Construction Done": 3,
        "Prior Construction": 3,
        "Operational": 4,
    }

    L = np.full((len(df), k), np.nan)
    for j in range(k):
        L[:, j] = pd.Series(S_sorted[:, j]).map(level_map).to_numpy(dtype=float)
        df[f"status_level_sorted_{j}"] = L[:, j]

    Ldf = pd.DataFrame(L)
    first_level = Ldf.bfill(axis=1).iloc[:, 0].to_numpy()
    last_level  = Ldf.ffill(axis=1).iloc[:, -1].to_numpy()

    df["status_level_first"] = first_level
    df["status_level_last"]  = last_level
    df["status_progress"]    = last_level - first_level
    df["status_level_max"]   = np.nanmax(L, axis=1)
    df["status_level_mean"]  = np.nanmean(L, axis=1)


    mask = ~pd.DataFrame(S_sorted).isna().to_numpy()
    n_changes = np.zeros(len(df), dtype=float)
    for j in range(k - 1):
        m = mask[:, j] & mask[:, j + 1]
        n_changes += (m & (S_sorted[:, j] != S_sorted[:, j + 1]))
    df["status_n_changes"] = n_changes

    n_reg = np.zeros(len(df), dtype=float)
    for j in range(k - 1):
        a, b = L[:, j], L[:, j + 1]
        m = np.isfinite(a) & np.isfinite(b)
        n_reg += (m & (b < a))
    df["status_n_regressions"] = n_reg
    df["status_has_regression"] = (n_reg > 0).astype(int)


    first_date = Di_sorted[:, 0]
    time_to_done = np.full(len(df), np.nan)
    for i in range(len(df)):
        if first_date[i] == maxv:
            continue
        idx_done = None
        for j in range(k):
            if valid_mask[i, j] and np.isfinite(L[i, j]) and L[i, j] >= 3:
                idx_done = j
                break
        if idx_done is not None:
            time_to_done[i] = (Di_sorted[i, idx_done] - first_date[i]) / (24 * 3600 * 1e9)
    df["time_to_done_days"] = time_to_done


    for color, stat, cols in img_specs:
        X = df[cols].to_numpy(dtype=float)
        Xs = np.take_along_axis(X, order[:, :k], axis=1)
        Xs = Xs.copy()
        Xs[~valid_mask] = np.nan

        first = Xs[:, 0]
        last = np.full(len(df), np.nan)
        ok = last_idx >= 0
        last[ok] = Xs[ok, last_idx[ok]]

        df[f"img_{color}_{stat}_first"] = first
        df[f"img_{color}_{stat}_last"]  = last
        df[f"img_{color}_{stat}_delta"] = last - first

        for j in range(k - 1):
            df[f"img_{color}_{stat}_delta_{j}_{j+1}"] = Xs[:, j + 1] - Xs[:, j]


    if drop_raw:
        drop_cols = []
        drop_cols += date_cols + status_cols
        for _, _, cols in img_specs:
            drop_cols += cols
        drop_cols += ["status_first", "status_last"]
        df = df.drop(columns=[c for c in drop_cols if c in df.columns])

    return df


In [9]:
#create temporal features for both train and test
x_train = add_temporal_status_image_features(x_train)
x_test = add_temporal_status_image_features(x_test) 

/var/folders/52/gw_8zl6n65b05xp9tk246k2m0000gn/T/ipykernel_61399/754418011.py:107: RuntimeWarning: All-NaN slice encountered
  df["status_level_max"]   = np.nanmax(L, axis=1)
/var/folders/52/gw_8zl6n65b05xp9tk246k2m0000gn/T/ipykernel_61399/754418011.py:108: RuntimeWarning: Mean of empty slice
  df["status_level_mean"]  = np.nanmean(L, axis=1)
/var/folders/52/gw_8zl6n65b05xp9tk246k2m0000gn/T/ipykernel_61399/754418011.py:107: RuntimeWarning: All-NaN slice encountered
  df["status_level_max"]   = np.nanmax(L, axis=1)
/var/folders/52/gw_8zl6n65b05xp9tk246k2m0000gn/T/ipykernel_61399/754418011.py:108: RuntimeWarning: Mean of empty slice
  df["status_level_mean"]  = np.nanmean(L, axis=1)


### Multi-hot encoding: urban_type & geography_type

In [10]:
def multi_hot_encode(df: pd.DataFrame, column: str, all_values: set = None) -> tuple:

    df = df.copy()
    

    if all_values is None:
        all_values = set()
        for val in df[column].dropna():
            if pd.notna(val) and val != 'N,A':  

                values = [v.strip() for v in str(val).split(',')]
                all_values.update(values)
    

    for value in sorted(all_values):

        col_name = f"{column}_{value.replace(' ', '_').replace('/', '_')}"
        

        df[col_name] = df[column].apply(
            lambda x: 1 if pd.notna(x) and value in str(x).split(',') else 0
        )
    

    df = df.drop(columns=[column])
    
    return df, all_values



def extract_all_values(df: pd.DataFrame, column: str) -> set:
    """Extrait toutes les valeurs uniques d'une colonne."""
    all_values = set()
    for val in df[column].dropna():
        if pd.notna(val) and val != 'N,A':
            values = [v.strip() for v in str(val).split(',')]
            all_values.update(values)
    return all_values


urban_values_train = extract_all_values(x_train, 'urban_type')
urban_values_test = extract_all_values(test_df, 'urban_type')
urban_values_all = urban_values_train | urban_values_test  

geo_values_train = extract_all_values(x_train, 'geography_type')
geo_values_test = extract_all_values(test_df, 'geography_type')
geo_values_all = geo_values_train | geo_values_test

In [11]:
#multi-hot encoding on train
x_train, _ = multi_hot_encode(x_train, 'urban_type', urban_values_all)
x_train, _ = multi_hot_encode(x_train, 'geography_type', geo_values_all)
print(f"Train shape: {x_train.shape}")

#multi-hot encoding on test
x_test, _ = multi_hot_encode(x_test, 'urban_type', urban_values_all)
x_test, _ = multi_hot_encode(x_test, 'geography_type', geo_values_all)
print(f"Test shape: {x_test.shape}")

Train shape: (296146, 95)
Test shape: (120526, 95)


### Drop additional columns

In [12]:
cols_to_drop = ["delta_sorted_0_1_days","delta_sorted_1_2_days", "delta_sorted_2_3_days", "delta_sorted_3_4_days","index"]


cols_to_drop_train = [col for col in cols_to_drop if col in x_train.columns]
cols_to_drop_test = [col for col in cols_to_drop if col in x_test.columns]

x_train = x_train.drop(columns=cols_to_drop_train)
x_test = x_test.drop(columns=cols_to_drop_test)

### Imputation

In [13]:
inf_count_train = np.isinf(x_train.select_dtypes(include=[np.number])).sum().sum()
inf_count_test = np.isinf(x_test.select_dtypes(include=[np.number])).sum().sum()
print(f"Train: {inf_count_train} infinity values")
print(f"Test: {inf_count_test} infinity values")

# Replace inf values with NaN for imputation
x_train = x_train.replace([np.inf, -np.inf], np.nan)
x_test = x_test.replace([np.inf, -np.inf], np.nan)

from sklearn.impute import SimpleImputer

numerical_cols_final = x_train.select_dtypes(include=["number", "bool"]).columns.to_list()
categorical_cols_final = x_train.select_dtypes(include=["object", "category"]).columns.to_list()

numerical_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Impute x_train
if len(numerical_cols_final) > 0:
    x_train[numerical_cols_final] = numerical_imputer.fit_transform(x_train[numerical_cols_final])
if len(categorical_cols_final) > 0:
    x_train[categorical_cols_final] = categorical_imputer.fit_transform(x_train[categorical_cols_final])

# Impute x_test 
if len(numerical_cols_final) > 0:
    x_test[numerical_cols_final] = numerical_imputer.transform(x_test[numerical_cols_final])
if len(categorical_cols_final) > 0:
    x_test[categorical_cols_final] = categorical_imputer.transform(x_test[categorical_cols_final])


Train: 2 infinity values
Test: 2 infinity values


# <u> NeuralNetwork Modele training <u/>

In [14]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score

In [18]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


X = x_train.values.astype(np.float32)
y = y_train.values.astype(np.int64)

# Define dimensions
in_dim = X.shape[1]
n_classes = len(np.unique(y))


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr_idx, va_idx = next(sss.split(X, y))
X_tr, y_tr = X[tr_idx], y[tr_idx]
X_va, y_va = X[va_idx], y[va_idx]

scaler = StandardScaler()
X_tr = scaler.fit_transform(X_tr).astype(np.float32)
X_va = scaler.transform(X_va).astype(np.float32)

train_ds = TensorDataset(torch.from_numpy(X_tr), torch.from_numpy(y_tr))
val_ds   = TensorDataset(torch.from_numpy(X_va), torch.from_numpy(y_va))

train_loader = DataLoader(train_ds, batch_size=2048, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=4096, shuffle=False, num_workers=0)


classes = np.unique(y_tr)
w = compute_class_weight(class_weight="balanced", classes=classes, y=y_tr)
class_w = torch.tensor(w, dtype=torch.float32)

def eval_f1(model, loader):
    model.eval()
    all_pred, all_true = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            logits = model(xb)
            pred = torch.argmax(logits, dim=1).cpu().numpy()
            all_pred.append(pred)
            all_true.append(yb.numpy())
    yhat = np.concatenate(all_pred)
    ytrue = np.concatenate(all_true)
    wF1 = f1_score(ytrue, yhat, average="weighted")
    mF1 = f1_score(ytrue, yhat, average="macro")
    return wF1, mF1

def train_with_early_stopping(model, optimizer, criterion,
                              max_epochs=50, patience=5, min_delta=1e-4):
    best_val = -1.0
    best_state = None
    bad = 0

    for epoch in range(1, max_epochs + 1):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

        tr_w, tr_m = eval_f1(model, train_loader)
        va_w, va_m = eval_f1(model, val_loader)

        print(
            f"Epoch {epoch:02d} | "
            f"Train wF1={tr_w:.4f} mF1={tr_m:.4f} | "
            f"Val wF1={va_w:.4f} mF1={va_m:.4f}"
        )

        if va_w > best_val + min_delta:
            best_val = va_w
            bad = 0
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
        else:
            bad += 1
            if bad >= patience:
                print("Early stopping.")
                break

    if best_state is not None:
        model.load_state_dict(best_state)

    best_tr_w, best_tr_m = eval_f1(model, train_loader)
    best_va_w, best_va_m = eval_f1(model, val_loader)

    return {
        "model": model,
        "best_train_wF1": best_tr_w,
        "best_train_mF1": best_tr_m,
        "best_val_wF1": best_va_w,
        "best_val_mF1": best_va_m,
    }

Model 1

In [16]:
'''class MLP1(nn.Module):
    def __init__(self, in_dim, n_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.25),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.25),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, n_classes),
        )

    def forward(self, x):
        return self.net(x)

print("\n" + "="*70)
print("TRAINING MODEL 1 (baseline)")
print("="*70)

model1 = MLP1(in_dim=in_dim, n_classes=n_classes).to(device)
criterion1 = nn.CrossEntropyLoss(weight=class_w.to(device))
optimizer1 = torch.optim.AdamW(model1.parameters(), lr=1e-3, weight_decay=1e-4)

res1 = train_with_early_stopping(
    model=model1,
    optimizer=optimizer1,
    criterion=criterion1,
    max_epochs=50,
    patience=5,
    min_delta=1e-4
)

print("\nMODEL 1 BEST SCORES")
print("Best TRAIN wF1:", res1["best_train_wF1"], "| Best TRAIN mF1:", res1["best_train_mF1"])
print("Best VAL   wF1:", res1["best_val_wF1"],   "| Best VAL   mF1:", res1["best_val_mF1"])
'''

'class MLP1(nn.Module):\n    def __init__(self, in_dim, n_classes):\n        super().__init__()\n        self.net = nn.Sequential(\n            nn.Linear(in_dim, 256),\n            nn.BatchNorm1d(256),\n            nn.ReLU(),\n            nn.Dropout(0.25),\n\n            nn.Linear(256, 128),\n            nn.BatchNorm1d(128),\n            nn.ReLU(),\n            nn.Dropout(0.25),\n\n            nn.Linear(128, 64),\n            nn.ReLU(),\n            nn.Linear(64, n_classes),\n        )\n\n    def forward(self, x):\n        return self.net(x)\n\nprint("\n" + "="*70)\nprint("TRAINING MODEL 1 (baseline)")\nprint("="*70)\n\nmodel1 = MLP1(in_dim=in_dim, n_classes=n_classes).to(device)\ncriterion1 = nn.CrossEntropyLoss(weight=class_w.to(device))\noptimizer1 = torch.optim.AdamW(model1.parameters(), lr=1e-3, weight_decay=1e-4)\n\nres1 = train_with_early_stopping(\n    model=model1,\n    optimizer=optimizer1,\n    criterion=criterion1,\n    max_epochs=50,\n    patience=5,\n    min_delta=1e-4\n)

Model 2

In [19]:
class MLP2(nn.Module):
    def __init__(self, in_dim, n_classes, dropout=0.10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, n_classes),
        )

    def forward(self, x):
        return self.net(x)

print("\n" + "="*70)
print("TRAINING MODEL 2 (higher capacity / less regularization)")
print("="*70)

model2 = MLP2(in_dim=in_dim, n_classes=n_classes, dropout=0.10).to(device)
criterion2 = nn.CrossEntropyLoss(weight=class_w.to(device))
optimizer2 = torch.optim.AdamW(model2.parameters(), lr=1e-3, weight_decay=1e-5)

res2 = train_with_early_stopping(
    model=model2,
    optimizer=optimizer2,
    criterion=criterion2,
    max_epochs=80,
    patience=7,
    min_delta=1e-4
)

print("\nMODEL 2 BEST SCORES")
print("Best TRAIN wF1:", res2["best_train_wF1"], "| Best TRAIN mF1:", res2["best_train_mF1"])
print("Best VAL   wF1:", res2["best_val_wF1"],   "| Best VAL   mF1:", res2["best_val_mF1"])


TRAINING MODEL 2 (higher capacity / less regularization)
Epoch 01 | Train wF1=0.6222 mF1=0.4341 | Val wF1=0.6212 mF1=0.4319
Epoch 01 | Train wF1=0.6222 mF1=0.4341 | Val wF1=0.6212 mF1=0.4319
Epoch 02 | Train wF1=0.6419 mF1=0.4432 | Val wF1=0.6396 mF1=0.4389
Epoch 02 | Train wF1=0.6419 mF1=0.4432 | Val wF1=0.6396 mF1=0.4389
Epoch 03 | Train wF1=0.5992 mF1=0.4224 | Val wF1=0.5981 mF1=0.4206
Epoch 03 | Train wF1=0.5992 mF1=0.4224 | Val wF1=0.5981 mF1=0.4206
Epoch 04 | Train wF1=0.6473 mF1=0.4519 | Val wF1=0.6443 mF1=0.4456
Epoch 04 | Train wF1=0.6473 mF1=0.4519 | Val wF1=0.6443 mF1=0.4456
Epoch 05 | Train wF1=0.6210 mF1=0.4293 | Val wF1=0.6203 mF1=0.4252
Epoch 05 | Train wF1=0.6210 mF1=0.4293 | Val wF1=0.6203 mF1=0.4252
Epoch 06 | Train wF1=0.6304 mF1=0.4336 | Val wF1=0.6285 mF1=0.4275
Epoch 06 | Train wF1=0.6304 mF1=0.4336 | Val wF1=0.6285 mF1=0.4275
Epoch 07 | Train wF1=0.6368 mF1=0.4489 | Val wF1=0.6319 mF1=0.4412
Epoch 07 | Train wF1=0.6368 mF1=0.4489 | Val wF1=0.6319 mF1=0.4412
Epoc